This is a Jupyter/Python version of Jeff Moser's [SQLite kernel](https://www.kaggle.com/jeffmoser/top-100-users-by-daily-submissions).  (It seems Kaggle no longer runs SQLite kernels directly.)  I just cut and pasted the query into my earlier notebook (see fork link at the top) and changed the data references and the output name.  Looks like that provides an easy template for running SQL queries.

In [ ]:
import numpy as np 
import pandas as pd
import sqlite3

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
path = "../input/"  #Insert path here
database = path + 'database.sqlite'

conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

In [ ]:
query = '''
WITH LatestSubmission AS (
   SELECT MAX(DateSubmitted) AS LastSubmissionDate  -- Get the day of the last recorded submission (i.e. last submission of a completed competition)
   FROM Submissions
),
TotalSubmissionsByUser AS (
    SELECT SubmittedUserId, COUNT(*) As TotalSubmissions
      FROM Submissions
  GROUP BY SubmittedUserId
),
UserDaysRegistered AS (
    SELECT u.Id,
           u.DisplayName,
           u.Ranking,
           (julianday(ls.LastSubmissionDate) - julianday(u.RegisterDate)) AS DaysRegistered
      FROM Users u, LatestSubmission ls
),
TotalDailySubmissionsByUser AS (
    SELECT (ts.TotalSubmissions / udr.DaysRegistered) AS DailySubmissions,
           udr.DisplayName, 
           udr.Ranking,
           udr.DaysRegistered
  FROM UserDaysRegistered udr
  JOIN TotalSubmissionsByUser ts ON  ts.SubmittedUserId = udr.Id
)
SELECT * 
 FROM TotalDailySubmissionsByUser
WHERE DaysRegistered > 100 -- Remove some noise by not including new users
ORDER BY DailySubmissions DESC
LIMIT 100
'''

In [ ]:
top100bySubs = pd.read_sql( query, conn )

In [ ]:
top100bySubs

In [ ]:
top100bySubs.DisplayName.values

In [ ]:
top100bySubs.to_csv("top100bySubs.csv", index=False)